In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from path import Path
from collections import Counter
# import s3fs
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import pycountry

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.ensemble import BalancedRandomForestClassifier

In [4]:
# Load the data
all_countries_df = pd.read_csv(Path("../Resources/all_countries.csv"), index_col=0)

all_countries_df

,Country,Year,GDP_per_capita,Population,Metric_Tons
0,Albania,2017,12770.963867,2884169.0,16481.0
1,Albania,2018,13317.092773,2882735.0,18094.0
2,Albania,2019,13656.592773,2880913.0,18500.0
3,Albania,2020,13191.952148,2877800.0,20157.0
4,Algeria,2017,11737.401367,41389174.0,153459.0
...,...,...,...,...,...
491,Uruguay,2020,21608.429688,3473727.0,110834.0
492,Vanuatu,2017,3081.835205,285499.0,868.0
493,Vanuatu,2018,3093.461426,292675.0,841.0
494,Vanuatu,2019,3137.240967,299882.0,997.0


In [5]:
all_countries_df.sort_values('Metric_Tons', ascending=False)

,Country,Year,GDP_per_capita,Population,Metric_Tons
79,China,2020,16315.815430,1.439324e+09,2777198.0
78,China,2019,15977.645508,1.433784e+09,2683611.0
77,China,2018,15133.996094,1.427648e+09,2498228.0
76,China,2017,14243.532227,1.421022e+09,2290405.0
371,Philippines,2020,7953.581543,1.095811e+08,1703846.0
...,...,...,...,...,...
248,Kiribati,2017,2235.707275,1.141530e+05,16.0
42,Belgium,2019,51944.167969,1.153933e+07,16.0
214,Iceland,2019,56935.773438,3.390370e+05,15.0
213,Iceland,2018,56818.195312,3.367120e+05,14.0


In [6]:
# all_countries_df['Metric_Tons'] = all_countries_df['Metric_Tons'].apply(lambda x: '%.5f' % x)

# all_countries_df

In [7]:
all_countries_bins_df = all_countries_df.dropna(how='all')
all_countries_bins_df

,Country,Year,GDP_per_capita,Population,Metric_Tons
0,Albania,2017,12770.963867,2884169.0,16481.0
1,Albania,2018,13317.092773,2882735.0,18094.0
2,Albania,2019,13656.592773,2880913.0,18500.0
3,Albania,2020,13191.952148,2877800.0,20157.0
4,Algeria,2017,11737.401367,41389174.0,153459.0
...,...,...,...,...,...
491,Uruguay,2020,21608.429688,3473727.0,110834.0
492,Vanuatu,2017,3081.835205,285499.0,868.0
493,Vanuatu,2018,3093.461426,292675.0,841.0
494,Vanuatu,2019,3137.240967,299882.0,997.0


In [8]:
all_countries_bins_df.describe()

,Year,GDP_per_capita,Population,Metric_Tons
count,492.000000,492.000000,4.920000e+02,4.920000e+02
mean,2018.500000,20689.958433,4.844674e+07,8.422285e+04
std,1.119172,18513.595598,1.794625e+08,2.956064e+05
min,2017.000000,1079.112427,1.780900e+04,9.000000e+00
25%,2017.750000,5450.180542,1.402112e+06,5.125000e+02
50%,2018.500000,14404.688477,7.674312e+06,5.878500e+03
75%,2019.250000,30073.445312,3.160235e+07,3.181250e+04
max,2020.000000,91738.750000,1.439324e+09,2.777198e+06


In [9]:
all_countries_bins_df = all_countries_bins_df.copy(deep=True)

In [10]:
#Put all Metric Tons values into quartile bins
all_countries_bins_df['Metric_Tons'] = pd.cut(all_countries_bins_df.Metric_Tons, bins=4,
       labels=['low', 'medium', 'high', 'extreme'])

In [11]:
all_countries_bins_df

,Country,Year,GDP_per_capita,Population,Metric_Tons
0,Albania,2017,12770.963867,2884169.0,low
1,Albania,2018,13317.092773,2882735.0,low
2,Albania,2019,13656.592773,2880913.0,low
3,Albania,2020,13191.952148,2877800.0,low
4,Algeria,2017,11737.401367,41389174.0,low
...,...,...,...,...,...
491,Uruguay,2020,21608.429688,3473727.0,low
492,Vanuatu,2017,3081.835205,285499.0,low
493,Vanuatu,2018,3093.461426,292675.0,low
494,Vanuatu,2019,3137.240967,299882.0,low


In [12]:
all_countries_bins_df = all_countries_bins_df.dropna(how='all')
all_countries_bins_df

,Country,Year,GDP_per_capita,Population,Metric_Tons
0,Albania,2017,12770.963867,2884169.0,low
1,Albania,2018,13317.092773,2882735.0,low
2,Albania,2019,13656.592773,2880913.0,low
3,Albania,2020,13191.952148,2877800.0,low
4,Algeria,2017,11737.401367,41389174.0,low
...,...,...,...,...,...
491,Uruguay,2020,21608.429688,3473727.0,low
492,Vanuatu,2017,3081.835205,285499.0,low
493,Vanuatu,2018,3093.461426,292675.0,low
494,Vanuatu,2019,3137.240967,299882.0,low


In [13]:
 # Create our features
target = ["Metric_Tons"]
X = all_countries_bins_df.drop(columns='Metric_Tons')
X = pd.get_dummies(X)

# Create our target
y = all_countries_bins_df.loc[:, target].copy()

In [14]:
 X.describe()

,Year,GDP_per_capita,Population,Country_Albania,Country_Algeria,Country_Angola,Country_Antigua and Barbuda,Country_Argentina,Country_Australia,Country_Bahamas,...,Country_Thailand,Country_Togo,Country_Tonga,Country_Trinidad and Tobago,Country_Tunisia,Country_Turkey,Country_Ukraine,Country_United Arab Emirates,Country_Uruguay,Country_Vanuatu
count,492.000000,492.000000,4.920000e+02,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,...,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000
mean,2018.500000,20689.958433,4.844674e+07,0.008130,0.008130,0.008130,0.008130,0.008130,0.008130,0.008130,...,0.008130,0.008130,0.008130,0.008130,0.008130,0.008130,0.008130,0.008130,0.008130,0.008130
std,1.119172,18513.595598,1.794625e+08,0.089891,0.089891,0.089891,0.089891,0.089891,0.089891,0.089891,...,0.089891,0.089891,0.089891,0.089891,0.089891,0.089891,0.089891,0.089891,0.089891,0.089891
min,2017.000000,1079.112427,1.780900e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2017.750000,5450.180542,1.402112e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2018.500000,14404.688477,7.674312e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2019.250000,30073.445312,3.160235e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2020.000000,91738.750000,1.439324e+09,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
 # Check the balance of our target values
y['Metric_Tons'].value_counts()

low        480
medium       4
high         4
extreme      4
Name: Metric_Tons, dtype: int64

In [16]:
 from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [17]:
 # Resample the training data with the BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

# train the model
brf.fit(X_train, y_train)

# get predictions
predictions = brf.predict(X_test)

In [18]:
BalancedRandomForestClassifier(random_state=1)

BalancedRandomForestClassifier(random_state=1)

In [19]:
# Calculated the balanced accuracy score
y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9385593220338984

In [20]:
 # Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[ 1,  0,  0,  0],
       [ 0,  3,  0,  0],
       [ 2, 11, 89, 16],
       [ 0,  0,  0,  1]], dtype=int64)

In [21]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

    extreme       0.33      1.00      0.98      0.50      0.99      0.99         1
       high       0.21      1.00      0.91      0.35      0.95      0.92         3
        low       1.00      0.75      1.00      0.86      0.87      0.74       118
     medium       0.06      1.00      0.87      0.11      0.93      0.88         1

avg / total       0.97      0.76      1.00      0.84      0.87      0.74       123



In [22]:
# Get importances and features
importances = brf.feature_importances_
cols = X.columns

# Store in a DataFrame
feature_importances_df = pd.DataFrame({'feature':cols, 'importance': importances})
feature_importances_df.head()

,feature,importance
0,Year,0.084000
1,GDP_per_capita,0.168000
2,Population,0.265333
3,Country_Albania,0.006000
4,Country_Algeria,0.000000


In [23]:
# Sort the DataFrame
feature_importances_df.sort_values('importance', ascending=False)

,feature,importance
2,Population,0.265333
1,GDP_per_capita,0.168000
56,Country_India,0.136667
22,Country_China,0.118000
94,Country_Philippines,0.113333
...,...,...
42,Country_Gabon,0.000000
40,Country_Finland,0.000000
39,Country_Fiji,0.000000
37,Country_Equatorial Guinea,0.000000


In [24]:
 # Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [25]:
EasyEnsembleClassifier(n_estimators=100, random_state=1)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [26]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9173728813559322

In [27]:
 # Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[ 1,  0,  0,  0],
       [ 0,  3,  0,  0],
       [ 0,  1, 79, 38],
       [ 0,  0,  0,  1]], dtype=int64)

In [28]:
 # Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

    extreme       1.00      1.00      1.00      1.00      1.00      1.00         1
       high       0.75      1.00      0.99      0.86      1.00      0.99         3
        low       1.00      0.67      1.00      0.80      0.82      0.65       118
     medium       0.03      1.00      0.69      0.05      0.83      0.71         1

avg / total       0.99      0.68      1.00      0.80      0.82      0.66       123

